In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 
import urllib.request
import json

! pip install --user tldextract

import tldextract

You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
def clear_url(origin_url):
    return tldextract.extract(origin_url).registered_domain

In [12]:
with urllib.request.urlopen('https://vendorlist.consensu.org/vendorlist.json') as response:
   vendorlist = json.loads(response.read().decode())

df_iab = json_normalize(vendorlist['vendors'])
df_iab['policyUrl'] = df_iab['policyUrl'].apply(clear_url)
df_iab.sort_values(by=['id'])

,featureIds,id,legIntPurposeIds,name,policyUrl,purposeIds
7,[],1,[],"Exponential Interactive, Inc",exponential.com,"[1, 2, 3, 4, 5]"
17,[2],2,[5],Captify Technologies Limited,captify.co.uk,"[1, 2, 3]"
62,[],3,[],affilinet,affili.net,"[2, 3, 4, 5]"
15,"[2, 3]",4,[],Roq.ad GmbH,roq.ad,"[1, 2, 3, 4, 5]"
8,[],6,[],AdSpirit GmbH,adspirit.de,"[1, 2, 3, 4, 5]"
16,[],7,[1],Vibrant Media Limited,vibrantmedia.com,"[2, 3, 4, 5]"
0,"[1, 2]",8,"[3, 5]",Emerse Sverige AB,emerse.com,"[1, 2, 4]"
3,"[1, 2, 3]",9,[],AdMaxim Inc.,admaxim.com,"[1, 2, 3, 4, 5]"
22,"[2, 3]",10,[3],"Index Exchange, Inc.",indexexchange.com,[1]
13,"[1, 2]",11,"[2, 3, 4, 5]",Quantcast International Limited,quantcast.com,[1]


In [4]:
df_google = pd.read_csv('https://storage.googleapis.com/adx-rtb-dictionaries/providers.csv')
df_google['policy_url'] = df_google['policy_url'].apply(clear_url)
df_google.head()

,provider_id,provider_name,policy_url,domains
0,3,2KDirect Inc.,ipromote.com,ad1.ipromote.com cdn.ipromote.com fra*.ipromot...
1,7,Amazon,amazon.co.uk,*eu-central-1.amazonaws.com *eu-west-1.amazona...
2,11,Acuityads,acuityads.com,acuityplatform.com cdn.acuityplatform.com clic...
3,12,LiveRamp,liveramp.com,idsync.rlcdn.com pippio.com rc.rlcdn.com rlcdn...
4,15,Adara Media,adara.com,adaraanalytics.com ag.yieldoptimizer.com cdnfs...


In [10]:
intersected_df = pd.merge(df_iab, df_google, how='left', left_on='policyUrl', right_on='policy_url')
clean_df = intersected_df[['id','name','policyUrl','provider_id','provider_name']]
clean_df.to_csv('./iab_google_mapping.csv',index=False)
clean_df

,id,name,policyUrl,provider_id,provider_name
0,8,Emerse Sverige AB,emerse.com,797.0,Emerse
1,12,BeeswaxIO Corporation,beeswax.com,1858.0,Beeswax
2,28,"TripleLift, Inc.",triplelift.com,933.0,TripleLift
3,9,AdMaxim Inc.,admaxim.com,827.0,AdMaxim
4,27,ADventori SAS,adventori.com,1248.0,Adventori
5,25,Oath (EMEA) Limited,oath.com,2777.0,Oath
6,26,Venatus Media Limited,venatusmedia.com,NaN,NaN
7,1,"Exponential Interactive, Inc",exponential.com,492.0,Exponential
8,6,AdSpirit GmbH,adspirit.de,85.0,AdSpirit AdServer
9,30,BidTheatre AB,bidtheatre.com,1167.0,Bidtheatre
